# I. Entendimento dos dados

## 0. Libraries and functions

### 0.1. Importing libraries

In [10]:
!pip install ucimlrepo

In [11]:
from ucimlrepo import fetch_ucirepo

import pandas as pd
import numpy as np
from scipy.stats import shapiro, kstest

### 0.2. Helper functions

In [12]:
# -----------------------------------------------------------------------------
# Análise descritiva completa para variáveis quantitativas
# Inclui estatísticas de dispersão, normalidade, outliers e qualidade dos dados
# -----------------------------------------------------------------------------

def _calc_distribution_stats(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calcula estatísticas de dispersão e forma para variáveis numéricas:
    - amplitude (range), assimetria (skewness), curtose (kurtosis)
    - coeficiente de variação (coef_var)
    """
    return pd.DataFrame({
        ('Distribution', 'range'): df.max() - df.min(),
        ('Distribution', 'skewness'): df.skew(),
        ('Distribution', 'kurtosis'): df.kurtosis(),
        ('Distribution', 'coef_var'): (df.std() / df.mean()).abs().replace(np.inf, np.nan)
    })

def _calc_normality_stats(df: pd.DataFrame, numeric_cols: pd.Index) -> pd.DataFrame:
    """
    Aplica testes de normalidade:
    - Shapiro-Wilk para amostras de 4 a 2000 elementos
    - Kolmogorov-Smirnov para amostras com mais de 2000 elementos
    Retorna os p-valores dos testes.
    """
    cols = [('Normality', 'shapiro_p'), ('Normality', 'ks_p')]
    normality = pd.DataFrame(index=numeric_cols, columns=pd.MultiIndex.from_tuples(cols), dtype=float)

    for col in numeric_cols:
        sample = df[col].dropna()
        n = len(sample)

        if 4 <= n <= 2000:
            normality.loc[col, ('Normality', 'shapiro_p')] = shapiro(sample)[1]
        elif n > 2000:
            normality.loc[col, ('Normality', 'ks_p')] = kstest(sample, 'norm', args=(sample.mean(), sample.std()))[1]

    return normality

def _calc_outlier_stats(df: pd.DataFrame) -> pd.DataFrame:
    """
    Detecta outliers com base no IQR (Intervalo Interquartílico).
    Retorna limites inferior/superior, número de outliers e presença de ao menos um outlier.
    """
    stats = df.describe(percentiles=[.25, .75]).T
    q1, q3 = stats['25%'], stats['75%']
    iqr = q3 - q1

    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    is_lower = df.lt(lower_bound)
    is_upper = df.gt(upper_bound)

    return pd.DataFrame({
        ('Outliers', 'has_outlier'): (is_lower | is_upper).any(),
        ('Outliers', 'n_lower'): is_lower.sum(),
        ('Outliers', 'n_upper'): is_upper.sum(),
        ('Outliers', 'IQR'): iqr,
        ('Outliers', 'lower_bound'): lower_bound,
        ('Outliers', 'upper_bound'): upper_bound
    })

def _calc_quality_stats(df: pd.DataFrame) -> pd.DataFrame:
    """
    Avalia qualidade dos dados:
    - número e porcentagem de valores ausentes
    - número de valores infinitos
    """
    return pd.DataFrame({
        ('Quality', 'has_missing'): df.isna().any(),
        ('Quality', 'n_missing'): df.isna().sum(),
        ('Quality', 'p_missing'): (df.isna().mean() * 100).round(2),
        ('Quality', 'n_inf'): np.isinf(df).sum()
    })

def describe_plus(df: pd.DataFrame) -> pd.DataFrame:
    """
    Gera uma descrição estatística expandida de um DataFrame numérico.
    Inclui estatísticas de dispersão, normalidade, outliers e qualidade dos dados.
    """
    numeric_cols = df.select_dtypes(include=np.number).columns

    parts = [
        _calc_distribution_stats(df),
        _calc_normality_stats(df, numeric_cols),
        _calc_outlier_stats(df),
        _calc_quality_stats(df)
    ]

    result = pd.concat(parts, axis=1)
    return result


## 1. Loading and understanding data

### 1.1. Loading data

In [13]:
# fetch dataset
auto_mpg = fetch_ucirepo(id=9)

# data (as pandas dataframes)
X = auto_mpg.data.features
y = auto_mpg.data.targets

# metadata
#print(auto_mpg.metadata)

# variable information
#print(auto_mpg.variables)

# Juntar X e y
df_raw = pd.concat([X, y], axis=1)
df_raw.head()

,displacement,cylinders,horsepower,weight,acceleration,model_year,origin,mpg
0,307.0,8,130.0,3504,12.0,70,1,18.0
1,350.0,8,165.0,3693,11.5,70,1,15.0
2,318.0,8,150.0,3436,11.0,70,1,18.0
3,304.0,8,150.0,3433,12.0,70,1,16.0
4,302.0,8,140.0,3449,10.5,70,1,17.0


### 1.2. Data description

#### 1.2.1 Numerical variables

Esta seção realiza uma **avaliação inicial da estrutura** dos dados para construção do dicionário, contendo:

**Análise geral dos dados**:
- **Dimensões** do Conjunto de Dados (volume de dados):
  - Linhas: Nº observações;
  - Colunas: Nº variáveis;
- **Tipos** de Dados e Metadados:
  - Tipos de variáveis (dtypes e info);
- Total de valores **faltantes**;
- Total de registros **duplicados**.

**Análise descritiva por coluna**:
- **Estatísticas básicas**:
  - Média, mediana, desvio padrão;
  - Valores mínimos e máximos;
  - Quartis (Q1, Q2, Q3);
- **Distribuição**:
  - Amplitude (range);
  - Assimetria (skewness);
  - Curtose (kurtosis);
  - Coeficiente de variação;
- Testes de **normalidade**:
  - Shapiro-Wilk (para amostras pequenas/médias: 4 ≤ n ≤ 2000);
  - Kolmogorov-Smirnov (para grandes amostras: n > 2000);
- Identificação de **outliers** (método IQR):
  - Limites inferior e superior;
  - Quantidade de outliers em cada cauda;
- **Qualidade** dos dados
  - Valores faltantes por coluna;
  - Valores infinitos por coluna.

In [14]:
# Data shape
print('Nº de linhas: {:,}'.format(df_raw.shape[0]))
print('Nº de colunas: {:,}'.format(df_raw.shape[1]))

# Total data missing
print('Nº total de registros faltantes: {:,}'.format(df_raw.isna().sum().sum()))

# Total data duplicated
print('Nº de registros duplicados: {:,}'.format(df_raw.duplicated().sum()))

Nº de linhas: 398
Nº de colunas: 8
Nº total de registros faltantes: 6
Nº de registros duplicados: 0


In [15]:
# Data info
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   displacement  398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   horsepower    392 non-null    float64
 3   weight        398 non-null    int64  
 4   acceleration  398 non-null    float64
 5   model_year    398 non-null    int64  
 6   origin        398 non-null    int64  
 7   mpg           398 non-null    float64
dtypes: float64(4), int64(4)
memory usage: 25.0 KB


In [16]:
df_raw.describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
displacement,398.0,193.43,104.27,68.0,104.25,148.5,262.00,455.0
cylinders,398.0,5.45,1.70,3.0,4.00,4.0,8.00,8.0
horsepower,392.0,104.47,38.49,46.0,75.00,93.5,126.00,230.0
weight,398.0,2970.42,846.84,1613.0,2223.75,2803.5,3608.00,5140.0
acceleration,398.0,15.57,2.76,8.0,13.82,15.5,17.18,24.8
model_year,398.0,76.01,3.70,70.0,73.00,76.0,79.00,82.0
origin,398.0,1.57,0.80,1.0,1.00,1.0,2.00,3.0
mpg,398.0,23.51,7.82,9.0,17.50,23.0,29.00,46.6


In [17]:
describe_plus(df_raw).round(2)

Distribution                            Normality       \
                    range skewness kurtosis coef_var shapiro_p ks_p   
displacement        387.0     0.72    -0.75     0.54      0.00  NaN   
cylinders             5.0     0.53    -1.38     0.31      0.00  NaN   
horsepower          184.0     1.09     0.70     0.37      0.00  NaN   
weight             3527.0     0.53    -0.79     0.29      0.00  NaN   
acceleration         16.8     0.28     0.42     0.18      0.04  NaN   
model_year           12.0     0.01    -1.18     0.05      0.00  NaN   
origin                2.0     0.92    -0.82     0.51      0.00  NaN   
mpg                  37.6     0.46    -0.51     0.33      0.00  NaN   

                Outliers                                                   \
             has_outlier n_lower n_upper      IQR lower_bound upper_bound   
displacement       False       0       0   157.75     -132.38      498.62   
cylinders          False       0       0     4.00       -2.00       14.00   
horsepower          True       0      10    51.00       -1.50      202.50   
weight             False       0       0  1384.25      147.38     5684.38   
acceleration        True       3       4     3.35        8.80       22.20   
model_year         False       0       0     6.00       64.00       88.00   
origin             False       0       0     1.00       -0.50        3.50   
mpg                 True       0       1    11.50        0.25       46.25   

                 Quality                            
             has_missing n_missing p_missing n_inf  
displacement       False         0      0.00     0  
cylinders          False         0      0.00     0  
horsepower          True         6      1.51     0  
weight             False         0      0.00     0  
acceleration       False         0      0.00     0  
model_year         False         0      0.00     0  
origin             False         0      0.00     0  
mpg                False         0      0.00     0

#### 1.2.2. Categorical variables

In [18]:
# Distribuição da variável 'origin'
(df_raw['origin']
    .value_counts(normalize=True)
    .reset_index(name='proportion')
    .sort_values('proportion', ascending=False)
    .round({'proportion': 3}))

,origin,proportion
0,1,0.626
1,3,0.198
2,2,0.176


In [32]:
# Distribuição da variável 'cylinders'
(df_raw['cylinders']
    .value_counts(normalize=True)
    .reset_index(name='proportion')
    .sort_values('proportion', ascending=False)
    .round({'proportion': 3}))

,cylinders,proportion
0,4,0.513
1,8,0.259
2,6,0.211
3,3,0.010
4,5,0.008


In [33]:
# Distribuição da variável 'model_year'
(df_raw['model_year']
    .value_counts(normalize=True)
    .reset_index(name='proportion')
    .sort_values('proportion', ascending=False)
    .round({'proportion': 3}))

,model_year,proportion
0,73,0.101
1,78,0.090
2,76,0.085
3,82,0.078
4,75,0.075
5,80,0.073
6,79,0.073
7,81,0.073
8,70,0.073
9,71,0.070


### 1.3. Data dictionary


| **FEATURE** | **DESCRIPTION** | **TYPE** | **INTERVALS** | **DETAILS** | **COMMENTS**
|---|---|---|---|---|---|
| `mpg` - **target** | Milhas por galão (consumo)               | Contínua | [9.0, 46.6]     | Presença de **outliers**                 | Maior = mais eficiente
| `displacement`     | Cilindrada do motor (in³)                | Contínua | [68, 455]       | Alta variabilidade                       |
| `cylinders`        | Número de cilindros                      | Ordinal  | {3, 4, 5, 6, 8} |                                          | (???) Confirmar natureza ordinal tb pode ser discreta
| `horsepower`       | Potência do motor (HP)                   | Contínua | [46, 230]       | Presença de **outliers** e **missings**  |
| `weight`           | Peso do veículo (lbs)                    | Contínua | [1613, 5140]    |                                          |
| `acceleration`     | Aceleração (mph em segundos)             | Contínua | [8.0, 24.8]     | Presença de **outliers**                 | Menor = mais rápido.
| `model_year`       | Ano do modelo                            | Discreta | [70, 82]        |                                          | Codificado como último dois dígitos (70 = 1970)
| `origin`           | Origem do veículo                        | Nominal  | {1, 2, 3}       | `1: 62.6%`, `2: 17.6%`, `3: 19.8%`       | (???) Sugere categorias, tipo 1=EUA, 2=Europa, 3=JapãoS

# II. Preparação dos dados